## Post-Voiced-Unvoiced Processing Tool

This tool is used to correct the "target algorithm" (Crepe).

It has buttons to:
1. Delete selected Crepe pitches when this voice is silent or on an unvoiced consonant
2. Correct Crepe pitches in the wrong octave, by selecting another algorithm's pitches in a range and moving Crepe to that algorithm's octave.
3. Undelete selected Crepe 0's to their values before pre-voiced-unvoiced processing. 

The tool automatically undeletes isolated singles and pairs of Crepe pitches to their pre-voiced-unvoiced values.

In [ ]:
import plotly.graph_objs as go
import plotly.offline as py
from plotly.subplots import make_subplots
import pandas as pd
import os
import plotly.io as pio
from plotly.offline import init_notebook_mode, iplot, plot
import math
from copy import deepcopy
import numpy as np
from IPython.display import display, clear_output
from ipywidgets import widgets, Button, HBox, VBox
import random



In [ ]:
def frequency_to_note(frequency):
    # define constants that control the algorithm
    NOTES = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'] # these are the 12 notes in each octave
    OCTAVE_MULTIPLIER = 2 # going up an octave multiplies by 2
    KNOWN_NOTE_NAME, KNOWN_NOTE_OCTAVE, KNOWN_NOTE_FREQUENCY = ('A', 4, 440) # A4 = 440 Hz

    # calculate the distance to the known note
    # since notes are spread evenly, going up a note will multiply by a constant
    # so we can use log to know how many times a frequency was multiplied to get from the known note to our note
    # this will give a positive integer value for notes higher than the known note, and a negative value for notes lower than it (and zero for the same note)
    note_multiplier = OCTAVE_MULTIPLIER**(1/len(NOTES))
    frequency_relative_to_known_note = frequency / KNOWN_NOTE_FREQUENCY
    distance_from_known_note = math.log(frequency_relative_to_known_note, note_multiplier)

    # round to make up for floating point inaccuracies
    distance_from_known_note = round(distance_from_known_note)

    # using the distance in notes and the octave and name of the known note,
    # we can calculate the octave and name of our note
    # NOTE: the "absolute index" doesn't have any actual meaning, since it doesn't care what its zero point is. it is just useful for calculation
    known_note_index_in_octave = NOTES.index(KNOWN_NOTE_NAME)
    known_note_absolute_index = KNOWN_NOTE_OCTAVE * len(NOTES) + known_note_index_in_octave
    note_absolute_index = known_note_absolute_index + distance_from_known_note
    note_octave, note_index_in_octave = note_absolute_index // len(NOTES), note_absolute_index % len(NOTES)
    note_name = NOTES[note_index_in_octave]
    return note_name

#### Note class

In [ ]:
debug = True

# Encapsulates a contiguous sequence of samples of similar pitches 
class Note:
    def __init__(self, sumFreq, numPitches):
        self.minSize = 7
        self.sumFreq = sumFreq
        self.numPitches = numPitches
        
    def update(self, freq, i):
        self.sumFreq += freq[i]
        self.numPitches += 1
    
    # Called after the last pitch of the Note is seen
    def set(self, freq, y, i, riseOrFall, setROF):
        lastNoteSet = False
        if debug:
            print(f"Setting note of length {self.numPitches} before time {i}")
        if self.numPitches >= self.minSize:
            if setROF:
                self.setRiseOrFall(freq, y, riseOrFall)
            else:
                self.discardPitches(freq, riseOrFall)
            self.setNote(freq, y, range(i-self.numPitches, i))
            lastNoteSet = True 
        else:
            self.discardPitches(freq, range(riseOrFall.start, i))
        self.reset(freq, i)
        return lastNoteSet
        
    def setRiseOrFall(self, freq, y, riseOrFall):
        if debug:
            print(f"Range ({riseOrFall.start}, {riseOrFall.stop}): rise or fall.")
        for i in riseOrFall:
            freq[i] = y[i]/2
    
    def setNote(self, freq, y, rng):
        frequency = self.sumFreq/self.numPitches
        if debug:
            print(f"Range ({rng.start}, {rng.stop}): {frequency}.")
        for i in rng:
            freq[i] = frequency
            
    def discardPitches(self, freq, rng):
        if debug:
            print(f"Range ({rng.start}, {rng.stop}): discarded.")
        for i in rng:
            freq[i] = 0
            
    def reset(self, freq, i):
        self.sumFreq = 0
        self.numPitches = 0
        
    


### Post-Processing utilities

In [ ]:
def isolated0s(a):
    return np.r_[False, (a[:-2] > 0) & (a[1:-1] == 0) &  (a[2:] > 0), False]

def isolated00s(a):
    first0s = (np.r_[False, False, (a[:-5] > 0) & (a[1:-4] > 0) & (a[2:-3] == 0)
              & (a[3:-2] == 0) & (a[4:-1] > 0) & (a[5:] > 0), False, False, False])
    return np.r_[False, first0s[1:] | first0s[:-1]]

# replace isolated 0's in z with notes in zref, and put the result into w
def fix0s(w, z, zref):
    np.putmask(w, isolated0s(z), zref)
    np.putmask(w, isolated00s(z), zref)


tolerance1 = 0.07
tolerance2 = 0.05
def inNbhd(p, q,tolerance = tolerance2):
    return (np.abs(p-q) <= (p*tolerance if p<q else q*tolerance))

### Pitches to Notes - Two versions

In [ ]:
# define "notes" and store them in freq:
# remove isolated 0's from y
# - for large enough a, if y[i+j] ~ y[i], j = 1, ..., a-1 but not a,  
#   the values of freq[i], freq[i+1], ..., freq[i+a-1] are set to the average of the y's
#   and i is set to i+a 
# - otherwise:
#   - i is saved to the current run of bad pitches, and i is set to i+1
#   - when the run ends, set freq[i] for each i in the run, to:
#     - freq[i]/2, if the run is monotone
#     - 0, otherwise
def setRun(frq, st, end):
    diffs = np.ediff1d(frq[max(st-1, 0):end+1]) # next - current
    if len(diffs) > 0 and (np.max(diffs) <= 0 or np.min(diffs) >= 0):
        for q in range(st, end):
            frq[q] = frq[q]/2
    else:
        for q in range(st, end):
            frq[q] = 0

def pitchesToNotes1(y, yref):
    yfix = np.array(y)
    fix0s(yfix, y, yref)
    diffs = np.ediff1d(yfix) # next - current
    isMax = np.r_[False, (diffs[1:]<0) & (diffs[:-1] >=0), False]
    isMin = np.r_[False, (diffs[:-1]<0) & (diffs[1:]>=0), False]
    freq = np.array(yfix).flatten()
    i = 0
    st = 0
    while i < len(freq):
        refFreq = freq[i]
        sumFreq = freq[i]
        a = 1
        while i+a < len(freq) and inNbhd(refFreq, freq[i+a], tolerance1):
            sumFreq += freq[i+a]
            if (isMin[i+a] and (freq[i+a] > refFreq)) or (isMax[i+a] and (freq[i+a] < refFreq)):
                refFreq = freq[i+a]
            a += 1

        if a > 6:
            for q in range(i,i+a):
                freq[q] = sumFreq/a
            setRun(freq, st, i)
            i += a
            st = i
        else:
            i += 1  
    return freq

In [ ]:
# define Notes and store them in freq: 
# remove isolated 0's from y
# as i is incremented,
# - y[i] is in the "neighborhood" of the nearest local min or max if it's within tolerance of their pitches
#   - if its in both neighborhoods the neighborhoods merge
#   - if its in neither, the previous neighborhood is complete and i belongs to a "rising or falling tone"
# - once a neighborhood is is complete:
#   - if it contains at least noteSize pitches it's a note:
#     freq[j] = avg of pitches in a note, for j in the note
#   - freq[j] = 0, otherwise
# - once y[i] is again in the neighborhood of a local min or max, 
#   the rising or falling tone is complete:
#   - if it is followed by a note:
#     freq[j] = y[j]/2, for j in the rising or falling tone
#   - freq[j] = 0, otherwise
#
# For now, reuses one Note object
def pitchesToNotes2(y, yref):
    
    # get local maxes and mins, etc.
    yfix = np.array(y)
    fix0s(yfix, y, yref)
    diffs = np.ediff1d(yfix) # next - current
    isMax = np.r_[False, (diffs[1:]<0) & (diffs[:-1] >=0), False]
    isMin = np.r_[False, (diffs[:-1]<0) & (diffs[1:]>=0), False]
    isMinOrMax = isMin | isMax
    diffs = np.append(diffs, [diffs[len(diffs)-1]])
    argMinOrMax = np.nonzero(isMinOrMax)[0]
    minOrMax = yfix[isMinOrMax]
    print("Min and Max Indices: ", argMinOrMax[5:15])
    print("Mins and Maxes: ", minOrMax[5:15])
    assert(y.shape == diffs.shape and y.shape == isMinOrMax.shape)
    freq = np.array(yfix).flatten()
    i = 0
    m = 0    
    # rise or fall to within tolerance of first local min or max
    while not inNbhd(yfix[i], minOrMax[m]):
        i += 1
    riseOrFall = range(i)
    
    # while i is less than the first local min or max
    note = Note(0, 0)
    while i < argMinOrMax[m]:
        note.update(freq, i) 
        i += 1    
    m += 1
    
    # while i is >= one local min or max and < the next,
    # state transtitions: InLastNbhd -> [[InBothNbhds | InRiseOrFall] -> InNextNbhd -> InLastNbhd
    #                                    | InNextNbhd -> InLastNbhd
    #                                    | InLastNbhd]
    state = "InLastNbhd"
    riseOrFallStart = 0
    lastNoteSet = False
    while m < len(minOrMax):
        print(f"i: {i}, m: {m}, last minOrMax: {minOrMax[m-1]}, next minOrMax: {minOrMax[m]}")
        while i < argMinOrMax[m]:
            # switch statement -- no two cases coexist, and in every case i->i+1
            # when we enter the statement, we are in the state at time i-1. 
            # we must transit to the state at time i
            # if a note ends at time i-1: at time i, call note.set(), which resets note
            # note.update() is called at the end of each case; the value is never used in state InRiseOrFall
            if state == "InLastNbhd":
                if inNbhd(yfix[i], minOrMax[m-1]):
                    if i == argMinOrMax[m-1]: # new note, no riseOrFall
                        lastNoteSet = note.set(freq, yfix, i, riseOrFall, lastNoteSet)
                        riseOrFall = range(i, i)                     
                    if inNbhd(yfix[i], minOrMax[m]):
                        state = "InBothNbhds" 
                elif inNbhd(yfix[i], minOrMax[m]): # new note, no riseOrFall
                    state = "InNextNbhd" 
                    lastNoteSet = note.set(freq, yfix, i, riseOrFall, lastNoteSet)
                    riseOrFall = range(i, i) 
                else:
                    state = "InRiseOrFall"
                    riseOrFallStart = i
                    lastNoteSet = note.set(freq, yfix, i, riseOrFall, lastNoteSet)
            elif state == "InBothNbhds":
                if i == argMinOrMax[m-1] and not inNbhd(yfix[i], minOrMax[m]):
                    state = "InLastNbhd"
                elif not inNbhd(yfix[i], minOrMax[m-1]):
                    state = "InNextNbhd"
            elif state == "InRiseOrFall":
                if i == argMinOrMax[m-1]: # new note, end of riseOrFall
                    note.reset(freq, i)
                    riseOrFall = range(riseOrFallStart, i+1)
                    state = "InLastNbhd" if not inNbhd(yfix[i], minOrMax[m]) else "InBothNbhds"
                elif inNbhd(yfix[i], minOrMax[m]): # new note, end of riseOrFall
                    state = "InNextNbhd"
                    riseOrFall = range(riseOrFallStart, i)
                    note.reset(freq, i)
            else: # InNextNbhd  
                    if i == argMinOrMax[m-1]:
                        state = "InLastNbhd" if not inNbhd(yfix[i], minOrMax[m]) else "InBothNbhds"   
            note.update(freq, i) 
            if debug and i in range(8480, 8500):
                print(f"i: {i}, yfix: {yfix[i]}, State: {state}, riseOrFall: {riseOrFall}, Num pitches: {note.numPitches}")
            i += 1 
        m += 1
    
    # e.g., drop to 0 in mid-note 
    if state == "InLastNbhd":
        lastNoteSet = note.set(freq, yfix, i, riseOrFall, lastNoteSet)
        
    # last rise or fall, typically final 0's
    note.setRiseOrFall(freq, yfix, range(i, len(freq)))
    return freq

pitchesToNotes = pitchesToNotes1

### Collections and Songs

In [ ]:
collections = {"sm":"Scherbaum Mshavanadze",
               "guria":"Teach Yourself Gurian Songs",
               "megrelia":"Teach Yourself Megrelian Songs"}

collection_directories = {"sm":
                          ["GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919",
                           "GVM017_ChvenMshvidobaTake2_Ozurgeti_ShalvaChemo2016_20160713",
                           "GVM019_DaleKojas_DidgoriVillage_Didgori_20160707",
                           "GVM031_EliaLrde_LakhushdiVillage_MuradGigoGivi_20160819",
                           "GVM097_KristeAghsdga_LakhushdiVillage_MuradGigoGivi_20160819"],
                          "guria":
                          ["Adila-Alipasha",
                           "Indi-Mindi",
                           'Mival Guriashi (1)' ,
                           'Pikris Simghera',
                           "Alaverdi",
                           "K'alos Khelkhvavi",
                           'Mival Guriashi (2)' , 
                           "Sabodisho",
                           "Khasanbegura",     
                           "Mok'le Mravalzhamieri",
                           'Sadats Vshobilvar',
                           "Beri Ak'vans Epareba", 
                           "Lat'aris Simghera",    
                           "Mts'vanesa Da Ukudosa", 
                           "Shermanduli",
                           "Brevalo",             
                           "Manana",         
                           'Nanina (1)',      
                           "Shvidk'atsa",
                           "Chven-Mshvidoba",    
                           "Maq'ruli",               
                           'Nanina (2)',          
                           'Supris Khelkhvavi',
                           'Didi Khnidan',     
                           "Masp'indzelsa Mkhiarulsa", 
                           "Orira",                
                           "Ts'amok'ruli",
                           "Gakhsovs, T'urpa",
                           "Me-Rustveli",        
                           "P'at'ara Saq'varelo"],
                         "megrelia":
                          ["Vojanudi Chkim Jargvals"]}

coll_key="guria"
directories = collection_directories[coll_key]

data_dir = "/Akamai/Voice/data/pitches-vuv-new/"
ref_data_dir = "/Akamai/Voice/data/pitches/"
working_coll = collections[coll_key]
larynx = False
working_song = "Ts'amok'ruli"

### Load songs into dictionary

In [ ]:
algos = ['boersma', 'noll', 'crepe', 'maddox', 'hermes', 'yin'] # Note: later add praat
data3 = {}
locations = {}

for algo in algos:
    data3[algo] = {}
    locations[algo] = {}
    for direct in directories:
        data3[algo][direct] = {
            #"mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        locations[algo][direct] = {
            #"mix": {},
            "bass": {},
            "middle": {},
            "top": {}
        }
        
def separate(adir):
    conv={}
    conv[0] = lambda s: float(s.strip() or 0)
    x,y = np.loadtxt(adir, unpack=True, usecols=(0,1), converters=conv)
    return (x,y)
    
def load_songs():
    global data_dir, working_coll, working_song
    for algorithm in sorted(os.listdir(data_dir)):
        if algorithm in data3:
            print(algorithm)
            for collection in sorted(os.listdir(f"{data_dir}{algorithm}")):
                if collection != working_coll:
                    continue
                print(" ", collection)
                sm = (collection == "Scherbaum Mshavanadze")
                suffix = 'ALRX' if larynx and sm else 'AHDS' 
                for song in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}")):
                    if (song != working_song):
                        continue
                    print("  ", song)
                    for location in sorted(os.listdir(f"{data_dir}{algorithm}/{collection}/{song}")):
                        if (location[-4:] == '.txt'):
                            x, y = separate(f"{data_dir}{algorithm}/{collection}/{song}/{location}")
                            xref, yref = separate(f"{ref_data_dir}{algorithm}/{collection}/{song}/{location}")
                            if (not np.array_equal(x, xref)):
                                print(f"    {location}: different x arrays before and after v-uv")

                            freq = pitchesToNotes(y, yref)
                            
                            notes = []
                            for i in range(0,len(freq)):
                                if freq[i] > 0:
                                    notes.append(frequency_to_note(freq[i]))
                                else:
                                    notes.append('N/A')

                            if suffix in location: #not suffix in location:
                               # data3[algorithm][song]['mix'] = (x, y, freq, yref)
                               # locations[algorithm][song]['mix'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                            #else:
                                #print(location)
                                name_tag = location[(location.index(suffix) + 3):(location.index(suffix) + 6)]
                                sm_kludge = sm and (not 'Batonebis' in location) # I renamed the Batonebis files :-p
                                if (name_tag[1] == '1' and not sm_kludge) or (name_tag[1] == '3' and sm_kludge):
                                    data3[algorithm][song]['bass'] = (x, y, freq, yref)
                                    locations[algorithm][song]['bass'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"

                                elif name_tag[1] == '2':
                                    data3[algorithm][song]['middle'] = (x, y, freq, yref)
                                    locations[algorithm][song]['middle'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"
                                else:
                                    data3[algorithm][song]['top'] = (x, y, freq, yref)
                                    locations[algorithm][song]['top'] = f"{data_dir}{algorithm}/{collection}/{song}/{location}"

    print("\nLoaded song data from files into dictionary")
                       
load_songs()                                
                                

### Graph the pitches

- Each algo:voice is a plotly "trace"
- In addition, there is a trace for the calculated "notes" of the target algo
- - This trace can't be selected or saved
- - After altering the target algo, re-graph to get the recalculated notes.

In [ ]:
selectedPoints = {}
targetSong = ""
traceIds = {}
targetAlgorithm = ""
#undoState = []
   
# can't select 
def selection_fn(trace,points,selector):
    global targetSong, selectedPoints
    splitArray = points.trace_name.split(": ")
    algoName = splitArray[0]
    if (algoName == f"{targetAlgorithm}Notes") or (algoName == f"{targetAlgorithm}Ref"):
        return
    voice = splitArray[1]
    if(len(trace.selectedpoints) != 0):                
        selectedPoints[algoName+"-"+voice] = trace.selectedpoints
    ##print(points)
    #print(trace.selectedpoints)
    
def graph(song, targetAlgo, reload=False):
    global targetSong, targetAlgorithm, undoState
    targetSong = song
    targetAlgorithm = targetAlgo
    print("target algo updated: " + targetAlgorithm)
    if reload:
        load_songs()
    print("plotting data from dictionary")
    traces = []
    type_list = []
    traceId = 0
    for algo, collection in data3.items():
        for name, part in collection.items():
            for audio_type, res in part.items():
                if name != song:
                    continue

                algolabel=algo
                try:
                    xp = res[0]
                    yp = res[1]
                    yref = res[3]
                    if algo == targetAlgorithm:
                        yp[yp==-1] = yref[yp==-1] # undo old, misguided postprocessing
                    trace = go.Scattergl(
                                x = xp,
                                y = yp,
                                name=f"{algolabel}: {audio_type}",
                                mode="markers",
                                visible= (True if audio_type == "bass" else False)
                            )

                    traceIds[algolabel+"-"+audio_type] = traceId
                    traceId += 1
                    traces.append(trace)
                    type_list.append(audio_type)
                    
                    if algo == targetAlgorithm:
                        algolabel=algo + "Notes"
                        trace = go.Scattergl(
                                    x = res[0],
                                    y = res[2],
                                    name=f"{algolabel}: {audio_type}",
                                    mode="markers",
                                    visible= (True if audio_type == "bass" else False)
                                )

                        traceIds[algolabel+"-"+audio_type] = traceId
                        traceId += 1
                        traces.append(trace)
                        type_list.append(audio_type)
                    
                    if algo == targetAlgorithm:
                        algolabel=algo + "Ref"
                        trace = go.Scattergl(
                                    x = res[0],
                                    y = res[3],
                                    name=f"{algolabel}: {audio_type}",
                                    mode="markers",
                                    visible= (True if audio_type == "bass" else False)
                                )

                        traceIds[algolabel+"-"+audio_type] = traceId
                        traceId += 1
                        traces.append(trace)
                        type_list.append(audio_type)
                        
                except:
                    print(f"{algolabel}: {audio_type} not available")

    layout = go.Layout(title='Activity Heatmap')

    figure = go.Figure(data=traces, layout=layout)

    fig = go.FigureWidget(figure)

    for i in range(0,len(traces)):
        fig.data[i].on_selection(selection_fn)

    buttons = []
    labels = ['bass', 'middle', 'top'] #, 'mix']
    for i, label in enumerate(labels):
        visibility = [label==current_type for current_type in type_list]
        button = dict(
            label = label,
            method = "update",
            args = [{ 'visible': visibility}]
        )

        buttons.append(button)

    updatemenus = list([
        dict(active=0,
            buttons=buttons,
            direction="down",
            pad={"r": 10, "t": 10},
            showactive=True,
            xanchor="left",
            yanchor="top",
            x = 0.005,
            y = 1.06,
        )
    ])

    fig.update_layout(
        legend=dict(
        orientation="h",
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
        ),
        margin=dict(l=0, r=0, t=100, b=0)
    )


    fig.update_traces(
        marker=dict(size=3),
        selector=dict(mode='markers')
    )


    fig['layout']['title'] = "Fixing " + targetAlgorithm + " for " + song
    fig['layout']['width'] = 900
    fig['layout']['height'] = 500
    fig['layout']['showlegend'] = True
    fig['layout']['updatemenus'] = updatemenus   
    
    undoState = deepcopy(fig.data)


    def update_axes(Octave):
        print("Correct octave updated: " + Octave)
        
        
    def on_button_octave(x):
        global targetAlgorithm, targetSong
        undoState = deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName != targetAlgorithm:
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                if len(selectedPoints[algo]) > 0:
                    selectedpoints = list(selectedPoints[algo])
                    if(len(selectedPoints[algo]) != 0):    
                        i = selectedPoints[algo][0]
                        #print(data3[algoName][targetSong][voice][2])
                        while True:
                            selectedpoints.append(i)
                            if data3[algoName][targetSong][voice][2][i-1] != data3[algoName][targetSong][voice][2][i]:
                                break;
                            i = i-1
                        i = selectedPoints[algo][-1]
                        while True:
                            selectedpoints.append(i)
                            if data3[algoName][targetSong][voice][2][i+1] != data3[algoName][targetSong][voice][2][i]:
                                break;
                            i = i+1
                    selectedPoints[algo] = tuple(selectedpoints)
                    for point in selectedPoints[algo]:
                        if fig.data[traceIds[targetAlgorithm+"-"+voice]].y[point] > 0 and fig.data[traceIds[algo]].y[point] > 0:
                            distances = list()
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-newData[point]))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*4)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/4)))

                            minimum = distances.index(min(distances))
                            if minimum == 0:
                                newData[point] = newData[point]
                            if minimum == 1:
                                newData[point] = newData[point]*2
                            if minimum == 3:
                                newData[point] = newData[point]*3
                            if minimum == 5:
                                newData[point] = newData[point]*4
                            if minimum == 2:
                                newData[point] = newData[point]/2
                            if minimum == 4:
                                newData[point] = newData[point]/3
                            if minimum == 6:
                                newData[point] = newData[point]/4
                            
                fig.data[traceIds[targetAlgorithm+"-"+voice]].y = newData
                
    def on_button_octave_selected(x):
        global targetAlgorithm, targetSong
        undoState = deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName != targetAlgorithm:
                newData = np.array(fig.data[traceIds[targetAlgorithm+"-"+voice]].y)
                if len(selectedPoints[algo]) > 0:
                    for point in selectedPoints[algo]:
                        if fig.data[traceIds[targetAlgorithm+"-"+voice]].y[point] > 0 and fig.data[traceIds[algo]].y[point] > 0:
                            distances = list()
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-newData[point]))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/2)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/3)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]*4)))
                            distances.append(abs(fig.data[traceIds[algo]].y[point]-(newData[point]/4)))

                            minimum = distances.index(min(distances))
                            if minimum == 0:
                                newData[point] = newData[point]
                            if minimum == 1:
                                newData[point] = newData[point]*2
                            if minimum == 3:
                                newData[point] = newData[point]*3
                            if minimum == 5:
                                newData[point] = newData[point]*4
                            if minimum == 2:
                                newData[point] = newData[point]/2
                            if minimum == 4:
                                newData[point] = newData[point]/3
                            if minimum == 6:
                                newData[point] = newData[point]/4
                            
                fig.data[traceIds[targetAlgorithm+"-"+voice]].y = newData

    def on_button_delete(x):
        undoState = deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            newData = np.array(fig.data[traceIds[algo]].y)
            if len(selectedPoints[algo]) != 0:
                for point in selectedPoints[algo]:
                    newData[point] = 0
            fig.data[traceIds[algo]].y = newData
    
    def on_button_delete_note(x):
        undoState = deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            location = locations[algoName][targetSong][voice]
            newData = np.array(fig.data[traceIds[algo]].y)
            if len(selectedPoints[algo]) != 0:
                selectedpoints = list(selectedPoints[algo])  
                i = selectedPoints[algo][0]
                print(data3[algoName][targetSong][voice][2])
                while True:
                    selectedpoints.append(i)
                    if data3[algoName][targetSong][voice][2][i-1] != data3[algoName][targetSong][voice][2][i]:
                        break;
                    i = i-1
                i = selectedPoints[algo][-1]
                while True:
                    selectedpoints.append(i)
                    if data3[algoName][targetSong][voice][2][i+1] != data3[algoName][targetSong][voice][2][i]:
                        break;
                    i = i+1
                selectedPoints[algo] = tuple(selectedpoints)
                for point in selectedPoints[algo]:
                    newData[point] = 0
                fig.data[traceIds[algo]].y = newData

    # use this button only if no other editing has been done
    # this button 
    # - starts with yfix, which is y (not the current trace) with isolated 0's fixed
    # - sets any pitches that don't belong to notes or between-note rises or falls to -1
    def on_button_postprocess(x):        
        global targetSong, targetAlgorithm
        undoState = deepcopy(fig.data)
        print("Target Algorithm post-processing starting...")
        #fig.restyle(fig, 'selectedpoints', null)
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            fullName = algoName+"-"+voice
            if algoName == targetAlgorithm:                
                yfix = np.array(data3[algoName][targetSong][voice][1])
                fix0s(yfix, data3[algoName][targetSong][voice][1], data3[algoName][targetSong][voice][3])
                #newData = np.array(fig.data[traceIds[algoName+"-"+voice]].y)
                frq = data3[algoName][targetSong][voice][2]
                for i in range(0, len(yfix)):
                    if frq[i] == 0 and yfix[i] != 0:
                        yfix[i] = -1                
                fig.data[traceIds[algoName+"-"+voice]].y = yfix
        print("Target Algorithm post-processing done.")
    
    def on_button_save(x):
        global targetSong
        print("Beginning saving files...")
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            if algoName == targetAlgorithm:
                location = locations[algoName][targetSong][voice]
                file = open(location, "w")
                for i in range(0,len(fig.data[traceIds[algoName+"-"+voice]].y)):
                    file.write(str(fig.data[traceIds[algoName+"-"+voice]].x[i])+" "+str(fig.data[traceIds[algoName+"-"+voice]].y[i]))
                    file.write("\n")
                file.close()
                print("Saved", location)
            
    def on_button_undo(x):
        global undoState
        for scatter in fig.data:
            splitArray = scatter.name.split(": ")
            algoName = splitArray[0]
            voice = splitArray[1]
            fullName = algoName+"-"+voice
            if algoName == targetAlgorithm:                
                newData = np.array(undoState[traceIds[algoName+"-"+voice]].y)          
                fig.data[traceIds[algoName+"-"+voice]].y = newData
    
    def on_button_deselect(x):
        global undoState
        for scatter in fig.data:                   
            fig.data[traceIds[algoName+"-"+voice]].selectedpoints = None
            fig.data[traceIds[algoName+"-"+voice]].selectedpoints = []

    def on_button_undelete(x):
        global targetAlgorithm, targetSong
        undoState = deepcopy(fig.data)
        for algo in selectedPoints:
            splitArray = algo.split("-")
            algoName = splitArray[0]
            voice = splitArray[1]
            if (algoName == targetAlgorithm):                
                pts = list(selectedPoints[algo])
                #print(f"Making {targetAlgorithm} a note in range {pts}")
                if len(pts) > 0:
                    algoRef = targetAlgorithm+"Ref-"+voice
                    yref = fig.data[traceIds[algoRef]].y
                    newData = np.array(fig.data[traceIds[algo]].y)
                    if np.amax(newData[pts]) <= 0:
                        newData[pts] = yref[pts]
                        fig.data[traceIds[algo]].y = newData
                        #print(f"Assigned pitch {np.average(y1[pts])} to range.")

    #button_postprocess = Button(description=f"Post-process {targetAlgorithm}", button_style='danger')
    button_octave = Button(description="Correct note", button_style='danger')
    button_octave_selected = Button(description="Correct selected", button_style='danger')
    button_delete = Button(description="Delete selected", button_style='danger')
    button_delete_note = Button(description="Delete note", button_style='danger')
    button_save = Button(description=f"Save {targetAlgorithm}", button_style='danger')
    button_undo = Button(description="Undo", button_style='danger')
    button_deselect = Button(description="Deselect", button_style='danger')
    button_undelete = Button(description=f"Undelete {targetAlgorithm}", button_style='danger')

    #button_postprocess.on_click(on_button_postprocess)
    button_octave.on_click(on_button_octave)
    button_octave_selected.on_click(on_button_octave_selected)
    button_delete.on_click(on_button_delete)
    button_delete_note.on_click(on_button_delete_note)
    button_save.on_click(on_button_save)
    button_undo.on_click(on_button_undo)
    button_deselect.on_click(on_button_deselect)
    button_undelete.on_click(on_button_undelete)

    #button_next.observe(on_button_next, )

    display(widgets.VBox([fig,HBox([button_delete, button_delete_note, button_octave_selected, button_octave]), 
                              HBox([button_save, button_undo, button_deselect, button_undelete])]))
    

### Can Correct

- Crepe (target algo) notes that should be 0 (usually another voice is singing)
- Crepe (target algo) notes for which another algorithm has a better estimate (in another octave)

### Cannot Correct

- consonants that show up as 0's or other notes (usually no algorithm has a good estimate)
- notes in the wrong octave but for which no algorithm has a good estimate in the right octave
- notes in the right octave that sound wrong 



In [ ]:
graph(working_song,'crepe', False)

#display(VBox([button_delete,button_next]))

## Problem sections

### Problems:

- partly-corrected notes
- 0's (probably from Voiced-Unvoiced algorithm) that should be notes

### Non-problems:

- consonants that show up as 0's or other notes
- notes that sound wrong but for which no algorithm offers a correction

#### Syntax

- start_time, algorithm, problem
- first_start_time-last_start_time, algorithm, problem (if there are 2 or more sections in succession)



### Scherbaum Mshavanadze songs


#### GVM009 - Bat'onebis Nanina - Larynx - no pitch-correction boxes, Bass&Middle&Top used.

Bass 

Middle - perfect

Top - perfect 


#### GVM009 - Bat'onebis Nanina - Headset - No voices used

I'm not going to correct this. I'll use Larynx.

Bass 

Middle

- 20.4, crepe deleted top note 3rd up on 'tkb'
- 46.5, crepe deleted top note 3rd up on 's'
- 54.7, crepe deleted top note 3rd up on 't'
- 62.1, crepe deleted top note 3rd up on 'sh'
- 77.0, crepe deleted top note 3rd up on 'ts'
- 91.8, crepe deleted bottom note on 'tqv'
- 118.3, crepe false 0, v-uv

Top - Yin is not registered, a little to the right, stopped deleting consonants except in obvious cases

- 14.1, crepe deleted consonant 's' (deleted already by v-uv?)

#### GVM017 - Chven Mshvidoba - Larynx - no pitch-correction boxes, No voices used.

Bass 

- deleted middle notes in most silent parts
- no pitch corrections

Middle 

- this mic picked up other voices

- deleted parts at 50Hz that should be 0
- 28.5, crepe deleted section of silence with non-obvious start and end times
- 41.5-42, corrected octave with Maddox
- 42.6-43.1, corrected octave with Maddox
- 43.7, crepe deleted section of silence with non-obvious start and end times
- 53.5, crepe deleted section of silence with non-obvious start and end times
- 56.3, crepe deleted section of silence with non-obvious start and end times
- 70.8, crepe deleted section of silence with non-obvious start and end times
- 78.0, crepe deleted section of silence with non-obvious start and end times
- 83.5, crepe deleted section of silence with non-obvious start and end times
- 89.4, crepe deleted section of silence with non-obvious start and end times
- 96.2, crepe deleted section of silence with non-obvious start and end times
- 101.6, crepe deleted section of silence with non-obvious start and end times
- 108.1-109.4, corrected octave with Noll
- 109.6, crepe deleted section of silence with non-obvious start and end times
- 127.5, crepe deleted section of silence with non-obvious start and end times
- 133.6, crepe deleted section of silence with non-obvious start and end times
- 140.1, crepe deleted section of silence with non-obvious start and end times
- 143.4, crepe deleted section of silence with non-obvious start and end times
- 150.6, crepe deleted section of silence with non-obvious start and end times
- 154.7, crepe deleted section of silence with non-obvious start and end times
- 157.7, crepe deleted section of silence with non-obvious start and end times
- 161.0, crepe deleted section of silence with non-obvious start and end times
- 165.6, crepe deleted section of silence with non-obvious start and end times

Top - almost perfect

- 40.4-40.9, corrected octave with Noll
- 166.2-166.6, corrected octave with Noll

#### GVM017 - Chven Mshvidoba - Headset - no pitch-correction boxes, Bass&Middle&Top used.

Bass 

- comparable to Larynx; there are a couple consonants Larynx doesn't drop
- no pitch corrections

Middle

- better than Larynx
- must not have fixed the octaves that Larynx got wrong, because there are no artifacts
- went through the Larynx silent parts and deleted some
- no pitch corrections

Top 

- better than Larynx
- must not have fixed the octaves that Larynx got wrong, because there are no artifacts
- no pitch corrections

#### GVM019 - Dale Kojas - Larynx - pitch-correction boxes, Bass&Middle used

Bass 

- better than Headset
- no pitch corrections

Middle 

- better than Headset
- pitch corrected

- 23.4-23.85, corrected octave with Yin, Maddox
- 30.7-31.2, corrected octave with Maddox
- 34.4-35.2, corrected octave with Maddox

Top - mostly false 0's, from v-uv?

#### GVM019 - Dale Kojas - Headset - no pitch-correction boxes, Top used

Bass 

- worse than Larynx; I think I deleted too many 'sh' consonants
- no pitch corrections

- removed all 'sh' consonants that are long and have high notes

Middle - top is too loud, need boxes

Top 

- better than Larynx
- no pitch corrections

- removed a few long 'sh' consonants

#### GVM031 - Elia Lrde - Larynx - no need for headset - no pitch-corrected boxes, Bass&Middle&Top used.

- no pitch correction

Bass - a few middle notes (below...?) deleted during silence

Middle - perfect

Top - perfect

#### GVM097 - Kriste Aghsdga - Larynx - no pitch-corrected boxes, Middle&Top used.

- no pitch corrections
- Larynx better for Middle, Top (tough call, I didn't correct anything for either)

Bass - mostly false 0's, from v-uv?

Middle - perfect

Top - perfect

#### GVM097 - Kriste Aghsdga - Headset - no pitch-corrected boxes,  Bass used.

- no pitch corrections
- Headset better for Bass
 
Bass - deleted middle notes during silence

Middle - also perfect

Top - also perfect



### Gurian songs A-I


#### Adila-Alipasha

Bass - 

Middle - added pitch-correction boxes

- Octaves corrected by Noll: 68, 74, 85

Top - added pitch-correction boxes

- 9.0-10.6, crepe corrected octaves with Hermes
- 16.4-17.3, crepe corrected octaves with Hermes
- 36.5-36.9, crepe corrected octave with Hermes
- 41-42, crepe corrected octave with Noll
- 49.2-49.4, crepe corrected octave with Noll, Boersma
- 49.4-53.2, crepe false 0's (fixed with yref), corrected octaves with Boersma
- 54.6-60.5, crepe false 0's (fixed with yref)
- 60.6-61.4, crepe corrected octave with Boersma
- 68.8-69.3, crepe false 0's (fixed with yref)
- 69, 71, crepe corrected octave with Boersma
- 73.8, 75.4, crepe corrected octave with Boersma
- 74.6-75.6, crepe false 0's (fixed with yref)
- 77.6-81.6, crepe false 0's (fixed with yref)
- 83.2, 85.2, 87, crepe corrected octave with Boersma
- 87.5, crepe false 0 (fixed with yref)
- 95.4, crepe corrected octave with Yin (correct, or wrong note?)
- 107.1, crepe false 0 after postprocessing (fixed with yref), Yin correct
- 107.1-110.5, crepe false 0's (fixed with yref)
- 110.7, crepe corrected octave with Boersma
- 117.1, crepe corrected octave with Boersma
- 117.7, crepe false middle/bass note ~5th below, Boersma correct
- 121.1-121.5, crepe false 0 after postprocessing (fixed with yref), Yin correct
- 130.9, crepe false 0 (fixed with yref)
- 133, crepe corrected octave with Boersma
- 134.9, crepe false 0 after postprocessing (fixed with yref), Boersma correct
- 135.44-135.49, crepe false 0 after postprocessing (fixed with yref), Boersma correct
- 135.6-135.9, crepe false 0 after postprocessing (fixed with yref), Yin correct
- 136.3, crepe corrected octave with Yin
- 136.6, 137.3, crepe false 0 after postprocessing (fixed with yref), Yin correct
- 137.6, crepe corrected octave with Yin
- 143.1, crepe corrected octave with Boersma


#### Alaverdi - added pitch-corrected boxes

Bass

Middle - crosses top a lot and is in 4-5 with top a lot

- 73.5, crepe deleted top note 3rd up on 't'

Top

- 119.8, crepe false 0, from v-uv, fixed


#### Beri Ak'vans Epareba

Bass

Middle - crosses top a lot

- 20.6-20.9, crepe deleted bass notes 3rd down on consonants
- 22.6, crepe deleted bass note 3rd down on consonant 's'
- 58.9, crepe deleted top note (below) on consonant 's'
- fixed: 77.1-78.3, crepe false 0 notes, v-uv
- 98.0, crepe deleted top note (crossing) on consonant 's'

Top

- 58.4, 59.3, crepe deleted false high notes on 's', 'ch'
- 96.9, 97.5, crepe deleted bottom notes on 's'


#### Brevalo

Bass

Middle

Top

#### Chven-Mshvidoba

Bass

Middle 

- 119.7, 122.8, crepe deleted bass notes on consonants 'ts'

Top - close to middle and crossing a lot


#### Didi Khnidan

No bass part

#### Gakhsovs, T'urpa

Bass

Middle

- 35.5, crepe deleted bottom note 3rd down on 'p'
- 72.9, crepe deleted top note 2nd up on 'sh'
- 73.9, crepe deleted bottom note 4th down on 'sh'
- fixed: 87.0-88.4, crepe false 0 notes, v-uv

Top - close to middle, hard to tells scoops from false notes

- 16.3, crepe deleted bottom note on 'rt'
- 44.1, crepe deleted middle note on 's'
- 45.2, crepe deleted bottom note on 'kv'
- 74.6, crepe deleted middle note on 'ts

#### Indi-Mindi

Bass

Middle

- 59.5, crepe false note 5th up (top), some algos partly correct
- 66.9-67.0, crepe deleted bottom note on 'kh'
- 87.3, crepe deleted bottom note on 'ls'
- 93.5, crepe false note 4th down
- 106.4-107.9, crepe deleted bottom notes on consonants, mostly 's'

Top

- 66.5, crepe deleted middle note on 'kh'
- 85.9-86.8, 88.1, crepe deleted middle notes on 'k', 'ts', 'ts'
- 100.5, crepe false note 6th high, no algo correct, looks like 'm'
- 105.4, 106.0, 107.2, crepe deleted middle notes on 's'
- fixed: 110, crepe false 0 on solo 'oi', v-uv


### Gurian songs K-M


#### K'alos Khelkhvavi

Bass

Middle - close to top on verses

- 12.26, crepe false bass note 4th down on 's', deleted
- fixed: 33.9-34.2, crepe false 0 notes from v-uv
- 56.5, 56.8, crepe deleted top notes on 'sh', 's'
- 64.0, 64.4 crepe deleted bottom notes on 'ch', 'sk'
- 68.3, crepe deleted bottom note on 's'
- 95.8, crepe deleted bottom note on 's'

Top

- 17.6, crepe octave fixed with Boersma
- 64.1, 64.6, crepe deleted middle notes 3rd up on 'chv', 'sk'
- 70.4, crepe deleted middle note 3rd up on 's'

#### Khasanbegura

Bass

- 26.2, false note 4th up on 'd', crepe deleted 
- 45.6, crepe octave corrected
- 48.2-48.4, "false" note from crepe choir overlap 4th below bass solo
- 75.2-75.5, crepe octave up corrected
- 85.6, crepe octave up corrected by Boersma
- note: 87.6-88.5, crepe choir overlap, bass solo mostly correct
- 125.9-126.4, false note from crepe choir 8th-9th up, no correct algo
- 166.0-166.7, false crepe choir notes, no correct algo
- 168.4-168.9, false crepe notes during choir overlap after bass solo entrance, no correct algo

Middle

- 8.5, 9.4, crepe false top/bass notes, Noll is correct
- 26.8, 27.2, crepe choir false bass notes, no correct algo
- 28.2, 28.6, crepe choir false bass note, Noll mostly correct but some parts uncorrected
- 29.3, 30.6, 31.7, crepe choir false bass notes, some algos correct on some parts
- 38.7, 39.9, 40.6, 41.5, 42.0, 42.3, 42.5, 42.8, 43.2, crepe choir false notes, mostly bass, no algos correct
- 47.9, 48.4, crepe solo deleted top/bass notes on 'ch', 'sh'
- 50.3, 51.9, crepe solo deleted top/bass notes on 'kh', '?'
- 53.7-53.9, crepe octave up corrected
- 67.6-87.1, false notes, crepe choir confused with bass on many notes
- 108.7-127.5, false notes, crepe choir confused with bass on many notes
- 119.3, crepe choir deleted top note on 's'
- 129.3, false note 3rd up on 'dzm', crepe deleted 
- 130.2-130.3, 130.8-130.9, crepe octaves corrected by Boersma
- 131.2, crepe solo deleted bass note on 'ch'
- 148.5-164.6, false notes, crepe choir confused with bass on many notes
- 172.3, 174.1, crepe solo deleted top/bass notes on 'ts', 'ch'
- 175.6-176.5, 177.9 crepe solo deleted top/bass notes 'k', 'ts', 'ts'
- 188.1-211.2, false notes, crepe choir confused with bass on many notes

Top - crepe is in the wrong octave a lot, regular and krimanchuli, Boersma correct 
 
- 15.2, 16.9 - false notes, crepe 4th up, Boersma correct
- 20.5, false notes, crepe wrong octave, no correct algo, tried to correct with Yin, made it worse
- 25.3-26.3, crepe wrong notes from choir 
- 62.1, crepe 1 octave high, only partly corrected by Hermes, Yin
- 67.4-69.5, crepe mostly 5th low, partly 4th high (bass choir), Boersma and Hermes mostly correct
- 108.2-109.1, crepe 1 octave high, parts uncorrected by Hermes, Yin
- 148.3-150.4, crepe mostly 5th low (bass choir), Boersma and Hermes mostly correct
- 188.0-189.9, crepe mostly 5th low (bass choir), Boersma and Hermes mostly correct
- 210.9-214.2, crepe mostly 5th low (bass choir), Boersma and Hermes mostly correct


#### Lat'aris Simghera

Bass

- 52.9, crepe deleted middle note on 'sh'
- 153.1, crepe deleted middle note on 's'
- 170.9, crepe deleted middle note on 's'
- 203.4, 203.9, false notes, crepe deleted, middle note 5th up, Noll is correct

Middle

- 84.8, crepe deleted bass note on 'sh'
- 104.1, crepe deleted bass note on 's'
- 148.7, crepe deleted top note on 'ch'
- 171.0, crepe deleted bass note on 'sh'
- 192.8, crepe deleted top note on 'ts'

Top

- 53.4, crepe deleted middle note? on 's'
- 92.9-94.0, crepe false middle notes, Noll correct an octave down
- 101.8, crepe false middle note, Noll correct
- 103.9, crepe deleted note on 's'
- 170.8 deleted middle note 1st down on 's'
- 215.2, crepe deleted middle/bass note on 's'


#### Manana

Bass

Middle

- 9.9, crepe deleted top/bottom note during 's'
- 14.7, crepe false top note 4th up
- 16.2, 16.8, crepe false bottom note 3rd down
- 70.0, crepe deleted bottom note during 's'
- 94.9, crepe deleted top note during 'kh'
- 104.6, crepe deleted bottom note during 'sh'
- 109.3, crepe deleted bottomnote on 't'

Top

- 24.5, crepe deleted middle note during 'kh'
- 69.8, crepe deleted middle note during 's'
- 76.7, crepe deleted middle note during 'sh'
- 109.0, crepe deleted middle note during 't'
- fixed: 116.8, crepe false 0 from v-uv

#### Maq'ruli

Bass

- 31.1, crepe false middle note, Noll correct
- 83.5, crepe false middle note, Boersma correct

Middle - hard to hear

- 15.6, crepe deleted bass note during 'ch'
- 16.5, crepe wrong octave, corrected?
- note: 19.1, crepe middle sings bass note for a moment
- 21.9-22.9, crepe, various wrong notes, Noll mostly correct
- 24.6, crepe false bass note, Noll correct
- 45.5-46.6, crepe, various wrong notes, Noll mostly correct
- 86.0, crepe false bass note, Noll correct
- 92.4, crepe false bass note, Noll correct
- 101.2-106.3, crepe, various wrong notes, Noll mostly correct
- 114.9-116.8, crepe, various wrong notes, Noll mostly correct
- 126.9-127.3, crepe, various wrong notes, Noll mostly correct
- 144.0-148.4, crepe, various wrong notes, Noll, Boersma, Hermes have some corrections

Top - might have called a lot of n's wrong notes

- 13.1, 13.3, crepe middle notes, Noll correct
- 20.7, crepe middle note, all voices wrong
- 25.0-26.4, crepe middle notes, Noll mostly correct
- 41.2-42.7, crepe wrong octave, no correct algo (some corrections on some notes)
- 49.7-50.0, crepe middle/bass note, other algos correct on parts of note
- 72.4-73.4, crepe various wrong notes, Noll, Boersma, Hermes have most of corrections
- 73.5, crepe old choir notes, misses new krimanchuli, Yin partly correct
- 95.6-96.8, crepe various wrong notes, Noll mostly correct
- 107.1, crepe middle note, Noll correct
- 112.2, crepe deleted middle note during 'n'
- 118.3, crepe middle note, Noll correct
- 122.8, crepe middle note, no correct algo
- 143.9, crepe middle note, Noll partly correct
- 149.5, crepe middle note, other algos have parts of correct note

#### Masp'indzelsa Mkhiarulsa

Bass

- 54.6-56.4, crepe deleted middle/top notes during 'ts, 'm', 's' (no correct algo on 'm')
- 77.8, crepe deleted middle note during 's'
- fixed: 105.5-107, crepe false 0 estimate, v-uv

Middle 

- 76.3-76.6, crepe deleted bass note during 'cht'

Top - crosses or is under middle a lot

- 17.4, crepe deleted note on 'st'


#### Me Rustveli

Bass

- 49.2, crepe deleted middle note during 's'
- 84.4, crepe deleted middle notes during 'ts'
- 117.8, crepe deleted middle notes on 'ts'
- 171.7, crepe octave up, corrected by Boersma
- 181.5-181.6, crepe deleted middle note during 'ks'
- 185.3, crepe deleted middle note during 'kv'

Middle

- 17.6, crepe deleted top note during 's'
- 51.1, crepe deleted bass note during 'ch?'
- 119.4, crepe deleted bass note during 'ts'

Top - crosses middle a lot, and takes the 4 in 1-4-5 chords

- 15.9-16.1, crepe deleted middle notes during consonants
- 83.2-84.1, crepe deleted middle notes during consonants
- 148.1-150.9, crepe deleted middle notes during consonants
- 167.8, 168.7, crepe false high note estimate, noll is correct
- 194.0-194.3, all algos confuse top and middle
- 197.6-197.8, crepe false 0 estimate, v-uv mistake

#### Mival Guriashi (1)

Bass

- 14.0, crepe deleted top note on 'sh'
- 64.5, crepe deleted middle note on 'ts'

Middle

- 13.9, crepe deleted top note during 'sh'
- 64.6, crepe deleted top note during 'kvsm'
- 80.3, crepe deleted top note on 't'
- 113.2, crepe deleted bass note during 'ts'

Top

- 13.8, crepe deleted middle note during 'sh'
- 32.1, crepe deleted middle notes during 'ts'
- 64.5, crepe deleted middle note on 'khts?'
- 81.8, crepe deleted bass note during 's'
- 95.7, 97.1 crepe deleted middle notes during 'kv', 's'


#### Mival Guriashi (2)

Bass

Middle

- 37.5, false note, crepe deleted 
- 39.0, crepe deleted top note on 'p?'
- 58.0, 58.3, 59.1, crepe deleted bass notes on 'rt', 'ch', 's'
- 105.5, 109.1, crepe deleted bass notes on 'sh', 'p'

Top

- 10.4, crepe deleted bass note on 'sh'
- 18.2, false note, crepe deleted note aug-5th down
- 18.8-19.2, false notes, crepe partly deleted bass note 8th down (no correct algos)
- 75.3, false note, crepe deleted bass note 8th down (no correct algos, voice cracking)
- 98.2-98.4, false note, crepe partly deleted bass note 8th down (no correct algos)


#### Mok'le Mravalzhamieri

Bass

Middle - crosses top a lot

Top

- 3.2, 4.0, 9.5, 10.1, 11.6, 17.4, 19.7 crepe estimates 4th or 5th too high (Noll mostly correct) 
- 18.7 crepe estimate 5th too high, miscorrected (Noll)

#### Mts'vanesa Da Ukudosa

Bass

Middle 

- 11.2, false note, crepe note deleted, noll correct

Top 

- 15.6, false note, crepe bass note deleted, noll correct
- note: 29.0, 32.2, 39.5, 72.4, 82.4, 102.8, 110.8, 113.5, 123.1, 126.3 crepe deleted mystery estimates during apparent silence
- 70.4, false note, crepe bass note
- 83.5, false note, crepe bass note (top note one octave up, all algorithms pick the bass note)
- 102.1, false note, crepe bass note



### Gurian songs N-T


#### Nanina (1)

Bass

- 65.1, false note, deleted crepe 4th up, no algo correct
- 69.4, crepe deleted middle note on 'k'

Middle

- 57.3, crepe deleted bottom/top notes on 'tkh'
- 66.9, 67.5, 68.2, crepe deleted top notes on 'kv', 'ts', 'tskh'
- 89.2-89.4, crepe octave down, corrected by Noll
- 92.2-92.6, crepe octave down, corrected by Boersma

Top

40.2-40.4, crepe looks like a corrected octave
41.1-41.3, crepe looks like a corrected octave
52.1-52.4, crepe looks like a corrected octave
64.8-65.1, crepe looks like a corrected octave

#### Nanina (2)

Bass

Middle

Top

- 5.5-5.8, crepe looks like a corrected octave
- 56.0, false note, crepe estimate deleted, another voice
- 66.75, false note, crepe bottom note deleted

#### Orira

Bass

- 82.2-82.4, false note, deleted crepe top note

Middle - hard to tell when the estimate is a bass note throughout; Noll better in some passages

- 20.1-20.5, false notes, deleted scattered crepe notes
- 21.4, false note, crepe 3rd down deleted
- 26.6-26.9, false notes, deleted scattered crepe notes
- 51.9-52.2, crepe deleted bottom note on 's'
- 65.3, 65.6, crepe deleted bottom notes on 's', 'k'
- 74.2, false note, crepe deleted bottom note
- 77.84-77.98, false note, crepe deleted bottom note
- 78.9, 79.2, crepe deleted bottom notes on 's', 'k'
- 79.84-80.06, crepe deleted bottom note on 's'
- 80.12-80.23, false note, deleted crepe bottom note
- 81.3, crepe deleted bottom note on 'sh'
- 96.2, crepe deleted bottom note on 'sh'
- 99.2-99.6, false note, crepe deleted bottom note, Noll correct
- 102.7, crepe deleted bottom note on 'sh'
- 105.6, crepe octave down, corrected by Noll
- 109.3, crepe deleted bottom note on 'sh'
- 112.3-112.4, false note, crepe deleted bass note, Noll correct
- 115.8, crepe deleted top note on 'sh'
- 122.4, crepe deleted top note on 'sh'
- 123.9-124.1, false note, crepe deleted top note, Noll correct
- 125.5-125.7, false note, crepe deleted bottom note, Noll correct
- 128.9, crepe deleted top note on 'sh'
- 133.6-133.9, false note 2nd down, crepe deleted, other algos 2nd up
- 135.5, crepe deleted top note on 'sh'
- 142.0, crepe deleted top note on 'sh'
- 143.7-144.0, false notes, crepe top note deleted, other algos wrong
- 143.4-143.5, false note, crepe top note deleted, other algos wrong
- 144.8, crepe octave down, corrected by Noll
- 149.8-150.1, false notes, crepe deleted bottom note, Noll partly correct
- 151.5-151.4, false notes, crepe deleted scattered notes, Noll partly correct on 2nd note
- 152.4-152.6, false notes, crepe deleted bass notes 3rd down
- 161.8-162.0, false notes, crepe deleted bass notes 3rd down
- 161.3-161-4, false notes, crepe deleted bass notes 3rd down
- 174.8-175.2, false notes, crepe deleted bass notes 3rd down
- 181.5-181.7, false notes, crepe deleted bass notes 3rd down

Top 

- 22.4, 24.3, 26.1, 29.7, crepe octave corrected 
- 32.8, 33.2, 33.5, crepe octave corrected
- 46.9, 47.8, 48.6, 49.5, crepe octave corrected
- 54.8, crepe octave corrected
- 59.2, 59.6, crepe octave corrected
- 74.5, 75.3, 76.2, crepe octave corrected
- 86.4, 87.3, 89.2, crepe octave corrected
- 94.0, 94.2, crepe octave corrected?
- 98.4, 99.5, crepe octave corrected
- 107.4, 111.5, crepe octave corrected
- 125.6, 133.5, crepe octave corrected
- 138.7, 139.1, crepe octave corrected
- 140.8, 142.0, 143.9, crepe octave corrected
- 156.7, 157.8, crepe octave corrected
- 161.3, 165.8, crepe octave corrected
- 170.1, 174.4, 178.9, 183.1, crepe octave corrected

- 21.3, false notes, deleted crepe scattered notes, no algo correct
- 41.9, false short note, deleted crepe scattered notes, no algo correct
- 46.2, false note, deleted crepe note, Yin correct
- 63.5, 63.6, false notes, deleted crepe notes, Yin 1st note correct, Yin octave up on 2nd 
- 77.6-77.7, false note, deleted crepe note, no algo correct
- 87.2, 87.8, 88.3, 89.3, false notes, Boersma mostly correct
- 94.4-94.5, false note, deleted crepe note, no algo correct
- 113.7-113.8, false note, deleted crepe note, no algo correct
- 121.1-121.4, false note, deleted crepe note, no algo correct
- 153.4-153.6, false note, deleted crepe note, no algo correct
- 158.1-158.2, false note, deleted crepe note, no algo correct

#### P'at'ara Saq'varelo - post-processed

Bass

Middle - hard to tell when the estimate is a bass note throughout

- 68.4, crepe consonant changed to 0 because estimate is bass note

Top 

- 31.0, false note, crepe deleted, Hermes correct

- places where octaves were corrected: 
 - 31.7-32.0, 
 - 49.0-49.3,
 - 75.4-76.0,
 

#### Pikris Simghera

Bass

Middle

Top

#### Sabodisho

Bass

- 106.7-107.3, crepe 1 octave low

Middle

- 86.1-87.0, false note, crepe 1 octave low, mostly corrected

Top


#### Shermanduli

Bass

Middle

Top

- 74.0-74.4, crepe octave corrected


#### Shvidk'atsa

Bass

- 61.8-62.5, false notes, crepe choir false notes, all algos
- 91.6-92.5, false notes, crepe choir, all algos

Middle

- 20.9-21.6, false note, crepe deleted, some algos partly correct
- 32.9-34.8, crepe deleted top/bass notes on consonants
- 59.6, false note, partly deleted crepe 2nd-3rd above, Noll partly correct
- 81.7-82.8, false notes, deleted crepe (1st-2nd choir change), Noll mostly, Hermes partly correct
- 89.4-89.7, false note, crepe mostly deleted, other algos mostly pick up bass above

Top

- 30.8-32.4, false notes, mostly deleted crepe, all algos partly correct
- 61.2-61.5, false note, crepe deleted 2nd above, Boersma mostly correct
- 62.6-62.7, false note, deleted crepe 3rd above, Boersma Maddox mostly correct
- 63.5, false notes, deleted crepe, all algos partly correct
- 91.97-92.2, false note, deleted crepe 3rd down, no algos correct


#### Supris Khelkhvavi

Bass

Middle

- 85.0, crepe deleted top note 5th up on 't'

Top

- 6.7-7.0, false note, deleted crepe middle, Hermes and Yin partly correct
- 39.9-40.0, false note, deleted crepe middle note (5th up), Noll correct
- 104.1, 105.3 crepe deleted middle note 5th down on 's'
- 129.3-129.5, crepe octave corrected by Noll
- 139.3, crepe deleted middle note 5th up on 'shl'
- 145.8, 146.6, crepe deleted middle notes on 'tsr', 'kh'


#### Ts'amok'ruli

Bass 

Middle

Top

- 2.6-3.1, crepe octave corrected
- 12.6-13.2, crepe octave corrected
- 22.8-23.3, crepe octave corrected
- 70.2-70.5, crepe octave corrected
- 12.6-13.2, crepe octave corrected
- 12.6-13.2, crepe octave corrected
- 73.7, crepe false middle note during short krimanchuli squawk, no algo correct


### Megrelian Songs

In [ ]:
x = np.array([1, 5, 3, 4])
print(np.max(x), np.min(x))
print(max(0, 2), min(0, 2))
print(x[0:5])

In [ ]:
yref = np.array([3,5,2,1,8,9,10])
y = np.array([3,0,-1,1,8,-1,10])
print(y)
y[y==-1] = yref[y==-1]
print(y)